# LightGCNPlus

### TODO: 
- try to learn the weighting of the different layers in the message passing mechanism.

### Scope
In this notebook we train different variants of our LightGCNPlus model on different hyperparameters. We then re-train the best performing model and evaluate it on Kaggle's public test set.

### About the model
LightGCNPlus is a graph-based collaborative filtering model that extends the [original LightGCN model](https://arxiv.org/pdf/2002.02126.pdf). The original LightGCN is only able to rank items. We thus refine the model to be able to predict ratings. Our LightGCNPlus model is composed of the following components:
- message passing: $e_u^{(l)} = \sum_{v \in N(u)} \frac{sr_{u,v}}{\sqrt{|N(u)||N(v)|}} e_v^{(l-1)}$, where $sr_{u,v}$ is the standardized rating of user $u$ for item $v$.
- aggregation mechanism: $h_u = \text{concat}(e_u^{(0)}, e_u^{(1)}, ..., e_u^{(L)})$, where $L$ is the number of layers in the message passing mechanism.
- output layer: $\hat{R}_{(i,j)} = \text{MLP}(\text{concat}(h_i, h_j))$, where $\text{MLP}$ is a multi-layer perceptron that projects the embedding couples of observed ratings to the output space.

How our model differs from the original GCMC model:
- In the message passing layer we use the layered message passing (multi-hop) mechanism of LightGCN instead of just a one-hop pass.
- To project to the output space we use a multi-layer perceptron instead of the softmax approach to compute probabilities over ratings.

### Hyperparameter tuning
The hyperparameters comprise both choices about the models' architecture and the training procedure.
We perform a grid search over the following hyperparameters:
...

Other hyperparameters such as the learning rate, the batch size, the number of epochs, the optimizer, etc. have been selected based on the results on previous experiments. Thus, to reduce the computational cost of the grid search, the following hyperparameters are kept constant:
- LR: learning rate.
- INIT_EMBS_STD: standard deviation of the normal distribution used to initialize the embeddings.
- WEIGHT_DECAY: weight decay coefficient for the Adam optimizer.
- DROPOUT: dropout rate used in the MLP's hidden layers.
- ACT_FN: activation function used in the MLP's hidden layers.

We also define hyperparameters for the training procedure:
- EPOCHS: number of backpropagation steps.
- STOP_THRESHOLD: minimum improvement in the validation loss to continue training.

### Training
- loss functions:
    - train: $\text{MSE} = \frac{1}{|\Omega|} \sum_{(i,j) \in \Omega} (M_{(i,j)} - \hat{M}_{(i,j)})^2$
    - eval: $\text{RMSE} = \sqrt{\text{MSE}}$
- optimizer: Adam
- Batching: no mini-batching of the training data, doesn't improve performance and slows convergence down. 
- Regularization: dropout and L2 regularization (weight decay in the Adam optimizer).

# Graph Convolutional Matrix Completion Plus (GCMCPlus)

### Scope
In this notebook we train different variants of our GCMCPlus model on different hyperparameters. We then re-train the best performing model and evaluate it on Kaggle's public test set.

### About the model
GCMCPlus is a graph-based collaborative filtering model that extends the [original GCMC model](https://arxiv.org/abs/1706.02263). We refine the model by taking ideas from the [neural collaborative filtering paper](https://arxiv.org/pdf/1708.05031.pdf) and the [LightGCN model](https://arxiv.org/pdf/2002.02126.pdf). Our GCMCPlus model is composed of the following components:
- preprocessing: for each rating level $r$, create $\tilde{A}_r = D_r^{-\frac{1}{2}} A_r D_r^{-\frac{1}{2}}$, where $A_r$ is the bipartite graph of the interactions of rating level $r$, and $D_r$ is the diagonal degree matrix of $A_r$.
- message passing: $e_{u_{r}}^{(l)} = \tilde{A}_{(u,v)_{r}} e_{v_{r}}^{(l-1)}$, where $e_{u_{r}}^{(0)}$ is the learnable embedding of user $u$ for rating level $r$.
- aggregation mechanism of layered message passing at rating level $r$: $h_{u_{r}} = \frac{1}{L} \sum_{l=1}^{L} e_{u_{r}}^{(l)}$, where $L$ is the number of layers in the message passing mechanism.
- aggregation mechanism of different rating levels: $h_u = \text{concat}(h_{u_1}, h_{u_2}, ..., h_{u_{|\mathcal{R}|}})$, where $|\mathcal{R}|$ is the number of rating levels.
- output layer: $\hat{M}_{(i,j)} = \text{MLP}(\text{concat}(h_i, h_j))$, where $\text{MLP}$ is a multi-layer perceptron that projects the embedding couples of observed ratings to the output space.

#### Architecture
1. Encoder:
    - create 5 bipartite graphs (one for each rating level)
    - for each node, message pass the embeddings of its neighbors
    - concatenate the embeddings of the different rating levels
    - pass the concatenated embeddings through a fully connected layer (decide whether to use act-MLP or just MLP)
2. Decoder:
    - compute probability of each rating level
    - compute final rating as expectation of the rating levels

#### Possible loss functions
- cross entropy: $-\sum_{(i,j) \in \Omega} \sum_{r \in \mathcal{R}} \text{I}\{M_{(i,j)} == r\} \log P(M_{(i,j)} == r)$
- RMSE: $\sqrt{\frac{1}{|\Omega|} \sum_{(i,j) \in \Omega} (M_{(i,j)} - \mathbb{E}[M_{(i,j)}])^2}$

In [1]:
import numpy as np
from sklearn.model_selection import train_test_split
import torch
from torch import nn

## 1. Load and Preprocess Data

In [2]:
from config import N_u, N_v, VAL_SIZE, DEVICE
from load import load_train_data
from preprocess import extract_users_items_ratings, create_bipartite_graph, create_degree_matrix, create_inverse_sqrt_degree_matrix

# Load data
train_df = load_train_data()

# Extract adjacency lists: observed values edge index (src, tgt) and ratings (values)
all_users, all_items, all_ratings = extract_users_items_ratings(train_df)

# Create rating matrix from the triplets
all_ratings_matrix = np.zeros((N_u, N_v))
all_ratings_matrix[all_users, all_items] = all_ratings

# Split the data into trai and val sets
train_users, val_users, train_items, val_items, train_ratings, val_ratings = \
    train_test_split(all_users, all_items, all_ratings, test_size=VAL_SIZE)

# convert lists to torch tensors
train_users = torch.tensor(train_users, dtype=torch.long).to(DEVICE)
val_users = torch.tensor(val_users, dtype=torch.long).to(DEVICE)
train_items = torch.tensor(train_items, dtype=torch.long).to(DEVICE)
val_items = torch.tensor(val_items, dtype=torch.long).to(DEVICE)
train_ratings = torch.tensor(train_ratings, dtype=torch.float).to(DEVICE)
val_ratings = torch.tensor(val_ratings, dtype=torch.float).to(DEVICE)

print(len(train_users) + len(val_users))

# Create adjacency lists for rating r 
u_v_r_train = []
u_v_r_val = []
total = 0
for r in range(1, 6):
    # assign value of 1 to each triplet with rating r instead of rating r
    u_v_r_train.append((train_users[train_ratings == r], train_items[train_ratings == r], torch.ones_like(train_ratings[train_ratings == r])))
    u_v_r_val.append((val_users[val_ratings == r], val_items[val_ratings == r], torch.ones_like(val_ratings[val_ratings == r])))
    total += len(train_users[train_ratings == r]) + len(val_users[val_ratings == r])
print(total)

# Create bipartite graphs
graphs_r_train = [create_bipartite_graph(u, v, r) for u, v, r in u_v_r_train]
graphs_r_val = [create_bipartite_graph(u, v, r) for u, v, r in u_v_r_val]

# check that graphs_r_train[0] doesn't contain only zero entries
mask1 = graphs_r_train[0] == 1
mask2 = graphs_r_train[1] == 1
mask3 = graphs_r_train[2] == 1
mask4 = graphs_r_train[3] == 1
mask5 = graphs_r_train[4] == 1

mask21 = graphs_r_val[0] == 1
mask22 = graphs_r_val[1] == 1
mask23 = graphs_r_val[2] == 1
mask24 = graphs_r_val[3] == 1
mask25 = graphs_r_val[4] == 1

total_entries = mask1.sum() + mask2.sum() + mask3.sum() + mask4.sum() + mask5.sum() + mask21.sum() + mask22.sum() + mask23.sum() + mask24.sum() + mask25.sum()
print(total_entries / 2)

# create degree matrix for each rating
degree_matrices_r = [create_degree_matrix(graph) for graph in graphs_r_train]
print(np.unique(degree_matrices_r[0].cpu().numpy()))
degree_norms_r = [create_inverse_sqrt_degree_matrix(degree_matrix) for degree_matrix in degree_matrices_r]
print(np.unique(degree_norms_r[0].cpu().numpy()))
# for each degree matrix, replace inf with 0
for i in range(5):
    degree_norms_r[i][degree_norms_r[i] == float('inf')] = 0
print(np.unique(degree_norms_r[0].cpu().numpy()))

# check that degree matrices are symmetric
print((degree_matrices_r[0] == degree_matrices_r[0].T).all())

# check that degree_norms_r[0] is symmetric
print((degree_norms_r[0] == degree_norms_r[0].T).all())

# create normalized adjacency matrices
norm_adj_r = [degree_norm @ graph @ degree_norm for degree_norm, graph in zip(degree_norms_r, graphs_r_train)]
print(np.unique(norm_adj_r[0].cpu().numpy()))

# send adj_matrices_r to device
norm_adj_r = [adj_matrix.to(DEVICE) for adj_matrix in norm_adj_r]

# Nice, everything works

1176952
1176952
tensor(1176952., device='mps:0')
[  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  97  98  99 100 101 102 103 104 106 107 108 109
 110 112 113 116 117 120 123 124 125 126 127 128 131 132 133 134 136 138
 142 144 146 149 155 166 167 168 173 174 184 188 191 197 198 209 217 246
 253 268 289 292 311 318 356]
[0.         0.0529999  0.05607722 0.0567048  0.05852057 0.05882353
 0.06108472 0.06286946 0.06375767 0.06788442 0.06917145 0.07106691
 0.07124705 0.07235746 0.0729325  0.07372098 0.07580981 0.07602859
 0.07715168 0.07738233 0.07761505 0.08032193 0.08192319 0.08276059
 0.08333334 0.08391814 0.08512565 0.0857493  0.08638684 0.086711
 0.0

In [3]:
class BaseLightGCN(nn.Module):
    def __init__(self, norm_adj_r, act_fn, embedding_dim, n_layers, init_emb_std, dropout_rate):
        super(BaseLightGCN, self).__init__()

        self.norm_adj_r = norm_adj_r  # bipartite graphs (one for each rating r)
        self.K = embedding_dim
        self.L = n_layers 
        self.act_fn = act_fn

        # Initialize embeddings
        self.E_u = nn.Embedding(num_embeddings=N_u, embedding_dim=self.K)
        self.E_v = nn.Embedding(num_embeddings=N_v, embedding_dim=self.K)
        nn.init.normal_(self.E_u.weight, std=init_emb_std)
        nn.init.normal_(self.E_v.weight, std=init_emb_std)

        # Projection to output space after message passing, aggregation, and selection
        self.mlp = self.create_mlp(dropout_rate)

        # crate learnable parameter list of Q_r matrices of shape K x K
        self.Q_r = nn.ParameterList([nn.Parameter(torch.randn(self.K, self.K)) for _ in range(5)])

    def create_mlp(self, dropout_rate):
        raise NotImplementedError("Derived classes must implement this method")
    
    def message_pass_r(self, r) -> list[torch.Tensor]:
        E_0 = torch.cat([self.E_u.weight, self.E_v.weight], dim=0)  # size (N_u + N_v) x K
        E_layers = [E_0]
        E_l = E_0

        for l in range(self.L):
            E_l = torch.mm(self.norm_adj_r[r], E_l)  # shape (N_u + N_v) x K
            E_layers.append(E_l) 

        # print("E_layers[0].shape: ", E_layers[0].shape)
        # print("len(E_layers): ", len(E_layers))
        return E_layers
    
    def aggregate_message_passing_r(self, E_r: list) -> torch.Tensor:
        E_agg_r = torch.stack(E_r, dim=0)  # shape (L + 1, N_u + N_v, K)
        E_agg_r_mean = torch.mean(E_agg_r, dim=0)  # shape (N_u + N_v, K)
        # print("E_agg_r.shape: ", E_agg_r.shape)
        # print("E_agg_r_mean.shape: ", E_agg_r_mean.shape)
        return E_agg_r_mean
    
    def aggregate(self, embs: list) -> torch.Tensor:
        """
        Aggregate the embeddings from the message passing layers.
        """
        E_agg = torch.cat(embs, dim=1)  # shape (N_u + N_v, K * (L + 1))
        # print("E_agg.shape: ", E_agg.shape)
        return E_agg
    
    def select_embeddings(self, users, items, E_agg):
        E_u, E_v = torch.split(E_agg, [N_u, N_v], dim=0)
        # Select embeddings of users and items from the adjacency lists
        E_u = E_u[users]
        E_v = E_v[items]  # shape (N_train, K * (L + 1))
        return E_u, E_v
    
    def forward(self, users, items):
        E_r = [self.message_pass_r(r) for r in range(5)]
        E_r_agg = [self.aggregate_message_passing_r(E) for E in E_r]
        E_agg = self.aggregate(E_r_agg)
        E_u_sel, E_v_sel = self.select_embeddings(users, items, E_agg)

        # Project to output space
        concat_users_items = torch.cat([E_u_sel, E_v_sel], dim=1)  # shape (N_train, (L + 1) * 5 * K * 2)
        out = self.mlp(concat_users_items).squeeze()  
        return out 

    def forward_1(self, users, items):
        # TODO: use lightGCN message passing to aggregate information over hops 
        E_r = [self.message_pass_r(r) for r in range(5)]
        # TODO: project down to K using MLP instead of aggregation
        E_agg = torch.mean(torch.stack(E_r), dim=0)
        E_u_sel, E_v_sel = self.select_embeddings(users, items, E_agg)

        # Compute logits for each rating level
        logits = []
        for r in range(5):
            logit = torch.einsum('ij,jk,ik->i', E_u_sel, self.Q_r[r], E_v_sel)
            logits.append(logit)

        logits = torch.stack(logits, dim=1)  # Shape: (batch_size, 5)

        # Compute the softmax probabilities
        softmax_probs = torch.softmax(logits, dim=1)  # Shape: (batch_size, 5)

        # Compute the final rating prediction as the expected value of the softmax probabilities
        ratings = torch.arange(1, 6).float().to(DEVICE)  # Shape: (5,)
        preds = torch.sum(softmax_probs * ratings, dim=1)  # Shape: (batch_size,)
        
        # assert all preds are in [1, 5]
        #assert (preds >= 1).all() and (preds <= 5).all()

        return preds

        

    def get_ratings(self, users, items):
        return self.forward(users, items)

class LightGCN(BaseLightGCN):
    def __init__(self, norm_adj_r, act_fn, embedding_dim, n_layers, init_emb_std, dropout_rate, projections):
        self.projections = projections
        super().__init__(norm_adj_r, act_fn, embedding_dim, n_layers, init_emb_std, dropout_rate)

        # For reproducibility after training
        # save_model_inputs(norm_adj_r, act_fn, embedding_dim, n_layers, init_emb_std, dropout_rate, projections)

    def create_mlp(self, dropout_rate):
        layers = []
        input_dim = self.K * 5 * 2
        for proj in self.projections:
            output_dim = self.K * proj
            layers.append(nn.Linear(input_dim, output_dim))
            layers.append(self.act_fn)
            layers.append(nn.Dropout(dropout_rate))
            input_dim = output_dim
        layers.append(nn.Linear(input_dim, 1))
        return nn.Sequential(*layers)

In [4]:
from train import train_model
from config import DEVICE
from train import train_model
from postprocess import report_training_results

# Model and optimizer hyperparameters
L=3
K=28
INIT_EMBS_STD=0.075
LR=0.1
WEIGHT_DECAY=0.00005
DROPOUT=0.5
PROJECTIONS = (5,)
ACT_FN = nn.GELU()
C=(5,)

# Train loop hyperparameters
EPOCHS=2000
STOP_THRESHOLD=1e-06

# to not change train loop (should actually separate concerns better and work with the reversing in the postprocessing)
means = np.zeros(N_v)
stds = np.ones(N_v)

In [5]:
model = LightGCN(norm_adj_r, ACT_FN, K, L, INIT_EMBS_STD, DROPOUT, C).to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr=LR, weight_decay=WEIGHT_DECAY)
loss_fn = nn.MSELoss()
print(f"K={K}, L={L}, C={C}")
train_rmse, val_rmse_std, val_rmse_orig = train_model(model, optimizer, loss_fn, train_users, train_items, train_ratings, val_users, val_items, val_ratings, val_ratings, means, stds, EPOCHS, STOP_THRESHOLD, False, verbosity=1)
report_training_results(train_rmse, val_rmse_std, val_rmse_orig)

K=28, L=3, C=(5,)
Epoch 1 - Avg loss in last 1 epochs: - Train: 5.9346 - Val std: 6.0958 - Val orig: 6.0958
Epoch 2 - Avg loss in last 1 epochs: - Train: 6.6244 - Val std: 59.1188 - Val orig: 59.1188
Epoch 3 - Avg loss in last 1 epochs: - Train: 59.6555 - Val std: 1.6401 - Val orig: 1.6401
Epoch 4 - Avg loss in last 1 epochs: - Train: 2.2601 - Val std: 5.1530 - Val orig: 5.1530
Epoch 5 - Avg loss in last 1 epochs: - Train: 5.3094 - Val std: 6.5373 - Val orig: 6.5373
Epoch 6 - Avg loss in last 1 epochs: - Train: 7.5576 - Val std: 8.9916 - Val orig: 8.9916
Epoch 7 - Avg loss in last 1 epochs: - Train: 10.7836 - Val std: 4.3794 - Val orig: 4.3794
Epoch 8 - Avg loss in last 1 epochs: - Train: 8.8006 - Val std: 10.0810 - Val orig: 10.0810
Epoch 9 - Avg loss in last 1 epochs: - Train: 13.1388 - Val std: 16.1591 - Val orig: 16.1591
Epoch 10 - Avg loss in last 1 epochs: - Train: 20.0191 - Val std: 38.8448 - Val orig: 38.8448
Epoch 11 - Avg loss in last 1 epochs: - Train: 40.6243 - Val std: 11.

KeyboardInterrupt: 